## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import matplotlib.pyplot as plt
import numpy as np

import time
from datetime import datetime

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

# Use the citipy module to determine city based on latitude and longitude.
from citipy import citipy

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,Weather_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Saint Pete Beach,US,27.7253,-82.7412,63.34,72,0,8.05,clear sky
1,1,Castro,BR,-24.7911,-50.0119,66.67,91,48,4.47,scattered clouds
2,2,Qaanaaq,GL,77.4840,-69.3632,-9.56,80,93,6.17,overcast clouds
3,3,Honningsvag,NO,70.9821,25.9704,19.54,86,100,7.25,light snow
4,4,Atuona,PF,-9.8000,-139.0333,78.08,77,19,12.91,light rain


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,Weather_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,Atuona,PF,-9.8000,-139.0333,78.08,77,19,12.91,light rain
5,5,Vaini,TO,-21.2000,-175.2000,86.16,74,40,13.80,scattered clouds
7,7,Bairiki,KI,1.3292,172.9752,86.00,70,20,9.22,few clouds
12,12,Rikitea,PF,-23.1203,-134.9692,78.62,71,66,12.82,broken clouds
16,16,Kapaa,US,22.0752,-159.3190,82.38,60,40,16.11,scattered clouds
17,17,Kavieng,PG,-2.5744,150.7967,81.99,77,97,6.24,moderate rain
18,18,Karratha,AU,-20.7377,116.8463,85.93,56,76,4.52,broken clouds
19,19,Quatre Cocos,MU,-20.2078,57.7625,81.09,85,87,14.56,light rain
38,38,Makakilo City,US,21.3469,-158.0858,79.61,47,0,11.50,clear sky
40,40,Butaritari,KI,3.0707,172.7902,80.80,79,68,12.53,moderate rain


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

Weather_ID             0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.count()

Weather_ID             188
City                   188
Country                188
Lat                    188
Lng                    188
Max Temp               188
Humidity               188
Cloudiness             188
Wind Speed             188
Current Description    188
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)

,Weather_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,Atuona,PF,-9.8000,-139.0333,78.08,77,19,12.91,light rain
5,5,Vaini,TO,-21.2000,-175.2000,86.16,74,40,13.80,scattered clouds
7,7,Bairiki,KI,1.3292,172.9752,86.00,70,20,9.22,few clouds
12,12,Rikitea,PF,-23.1203,-134.9692,78.62,71,66,12.82,broken clouds
16,16,Kapaa,US,22.0752,-159.3190,82.38,60,40,16.11,scattered clouds
...,...,...,...,...,...,...,...,...,...,...
705,705,Coronado,US,32.6859,-117.1831,89.82,15,0,12.66,clear sky
707,707,Ilhabela,BR,-23.7781,-45.3581,75.13,89,50,9.22,scattered clouds
709,709,Sahuaripa,MX,29.0500,-109.2333,80.64,13,19,5.99,few clouds
711,711,Bac Lieu,VN,9.2850,105.7244,79.66,79,28,15.43,scattered clouds


In [8]:
clean_df = preferred_cities_df.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)

In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Atuona,PF,78.08,light rain,-9.8000,-139.0333,
5,Vaini,TO,86.16,scattered clouds,-21.2000,-175.2000,
7,Bairiki,KI,86.00,few clouds,1.3292,172.9752,
12,Rikitea,PF,78.62,broken clouds,-23.1203,-134.9692,
16,Kapaa,US,82.38,scattered clouds,22.0752,-159.3190,
17,Kavieng,PG,81.99,moderate rain,-2.5744,150.7967,
18,Karratha,AU,85.93,broken clouds,-20.7377,116.8463,
19,Quatre Cocos,MU,81.09,light rain,-20.2078,57.7625,
38,Makakilo City,US,79.61,clear sky,21.3469,-158.0858,
40,Butaritari,KI,80.80,moderate rain,3.0707,172.7902,


In [12]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        
hotel_df    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Atuona,PF,78.08,light rain,-9.8000,-139.0333,Villa Enata
5,Vaini,TO,86.16,scattered clouds,-21.2000,-175.2000,Keleti Beach Resort
7,Bairiki,KI,86.00,few clouds,1.3292,172.9752,Tarawa Boutique Hotel
12,Rikitea,PF,78.62,broken clouds,-23.1203,-134.9692,People ThankYou
16,Kapaa,US,82.38,scattered clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
...,...,...,...,...,...,...,...
705,Coronado,US,89.82,clear sky,32.6859,-117.1831,El Cordova Hotel
707,Ilhabela,BR,75.13,scattered clouds,-23.7781,-45.3581,Ilha Flat Hotel
709,Sahuaripa,MX,80.64,few clouds,29.0500,-109.2333,Estatua de Sahuaripa
711,Bac Lieu,VN,79.66,scattered clouds,9.2850,105.7244,Khách sạn Sài Gòn Bạc Liêu


In [13]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.isnull().sum()

City                   0
Country                0
Max Temp               0
Current Description    0
Lat                    0
Lng                    0
Hotel Name             0
dtype: int64

In [14]:
clean_hotel_df = hotel_df.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)

In [20]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_search/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [21]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [22]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))